In [1]:
import tvm
from tvm import relay
from tvm.relay.testing import run_infer_type
from tvm.relay.transform import gradient
from tvm.relay import create_executor
import numpy as np

In [2]:
def show_gradient():   
    shape = (2, 2)
    dtype = 'float32'
    t = relay.TensorType(shape, dtype)
    x = relay.var("x", t)
    func = relay.Function([x], x + x)
    func = run_infer_type(func)
    back_func = run_infer_type(gradient(func))
    assert back_func.checked_type == relay.FuncType([t], relay.TupleType([t, relay.TupleType([t])]))
    ex = create_executor()
    x = tvm.nd.array(np.random.rand(*shape).astype(dtype))
    print(x)
    forward, (grad,) = ex.evaluate(back_func)(x)
    print(forward)
    print(grad)
    tvm.testing.assert_allclose(forward.asnumpy(), 2 * x.asnumpy())
    tvm.testing.assert_allclose(grad.asnumpy(), 2 * np.ones_like(x.asnumpy()))

In [3]:
show_gradient()

[[0.2826777  0.9061723 ]
 [0.7054706  0.94756633]]
[[0.5653554 1.8123446]
 [1.4109412 1.8951327]]
[[2. 2.]
 [2. 2.]]
